In [19]:
from pyaxis import pyaxis
import pandas as pd
#pd.options.mode.chained_assignment = None
from collections import defaultdict

Read in CSO dataset from the pc axis file using pyaxis.parse().

In [2]:
px = pyaxis.parse('C:/Users/ronan/PycharmProjects/Irish_Airport_Moverment/CTM01.px', encoding='ISO-8859-2')
df = pd.DataFrame(px['DATA'])


The CSO dataset has 5 columns recording the number of people travelling to and from irish airports each month from January 2006 to April 2020. A total of 3,905,603 entries in this dataset.

In [3]:
df

,Direction,Irish Airport,Foreign Airport,Month,DATA
0,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M01,0
1,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M02,0
2,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M03,0
3,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M04,0
4,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M05,0
...,...,...,...,...,...
3905599,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2019M12,NaN
3905600,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M01,NaN
3905601,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M02,NaN
3905602,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M03,NaN


The longitude and Latitude values for each foreign airport are available in the 'GlobalAirportDatabase' text file I have in the project folder. To reference each airport I will need the IATA(International Air Transport Association) code. This can be found as part of the string in the 'Foreign Airport' column. 

The entries in 'Foreign Airport' are consistant and therefore we can apply a parser function to each row to parse the string between the parentheses. 

In [4]:
def parser(str1):
    return str1[str1.find("(")+1:str1.find(")")]

Add a new column 'Airport_code' containg the parsed IATA code for each airport

In [5]:
df['Airport_code'] = df.apply(lambda row : parser(row['Foreign Airport']), axis = 1) 

In [6]:
df.head()

,Direction,Irish Airport,Foreign Airport,Month,DATA,Airport_code
0,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M01,0,AAH
1,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M02,0,AAH
2,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M03,0,AAH
3,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M04,0,AAH
4,All directions,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M05,0,AAH


The Direction column has 3 possible values, 'Inward', 'outward' and 'All directions'. We can filter out 'All directions' as this is not used and accounts fo 33% of the dataset.

In [8]:
df['Direction'].unique()

array(['All directions', 'Inward', 'Outward'], dtype=object)

Filter out the all directions entries to reduce the size of the dataframe

In [9]:
traffic_df = df.copy()
traffic_df = traffic_df[(traffic_df['Direction'] == 'Outward') | (traffic_df['Direction'] == 'Inward')]

Read in reference dataset with airport locations. Remove unwanted columns. 

In [10]:
locations_df=pd.read_csv("GlobalAirportDatabase.txt", sep=':',header =None,names=['ICAO Code','IATA Code','Airport Name','City/Town','Country','Latitude Degrees','Latitude Minutes','Latitude Seconds','Latitude Direction','Longitude Degrees','Longitude Minutes','Longitude Seconds','Longitude Direction','Altitude','Latitude Decimal Degrees','Longitude Decimal Degrees'])
refined_locations_df = locations_df[['IATA Code','Country','Latitude Decimal Degrees','Longitude Decimal Degrees']].copy()

In [11]:
refined_locations_df

,IATA Code,Country,Latitude Decimal Degrees,Longitude Decimal Degrees
0,GKA,PAPUA NEW GUINEA,-6.082,145.392
1,LAE,PAPUA NEW GUINEA,0.000,0.000
2,MAG,PAPUA NEW GUINEA,-5.207,145.789
3,HGU,PAPUA NEW GUINEA,-5.826,144.296
4,LAE,PAPUA NEW GUINEA,-6.570,146.726
...,...,...,...,...
9295,NaN,CHINA,0.000,0.000
9296,DLC,CHINA,38.966,121.538
9297,NaN,CHINA,0.000,0.000
9298,NaN,CHINA,0.000,0.000


Clean the dataset by removing the 'NaN' and 0 entries. Once cleaned, we can create our reference dictionary that is used to merge the 2 datasets. The reference dictionary will be nested with the IATA code as the key to the Country, Latitude and Longitude of each airport.

For example: { 'STN' :

                     {'Country': 'ENGLAND',
                     'Latitude Decimal Degrees': 51.885,                
                     'Longitude Decimal Degrees': 0.235} 
            }

In [12]:
refined_locations_df=refined_locations_df.dropna()
refined_locations_df = refined_locations_df[refined_locations_df["Latitude Decimal Degrees"]!=0]
d = defaultdict(dict)
for i, row in refined_locations_df.iterrows():
    d[row['IATA Code']] = row.drop(['IATA Code']).to_dict()

reference_locations_dict = dict(d)

In [13]:
reference_locations_dict['STN']

{'Country': 'ENGLAND',
 'Latitude Decimal Degrees': 51.885,
 'Longitude Decimal Degrees': 0.235}

There was some airports present in the CSO data that did not have a reference location from the Global Airport Database. These airports were mainly tiny airfields that we can just neglect for this purpose. We can make a list of the know locations only and filter the CSO dataframe so that only the airports with known locations are included.

In [14]:
unique_codes = traffic_df['Airport_code'].unique()
unknown_locations = [code for code in unique_codes if code not in reference_locations_dict]
known_df = traffic_df[~traffic_df['Airport_code'].isin(unknown_locations)]
known_df = known_df.reset_index()

In [15]:
known_df

,index,Direction,Irish Airport,Foreign Airport,Month,DATA,Airport_code
0,1301868,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M01,0,AAH
1,1301869,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M02,0,AAH
2,1301870,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M03,0,AAH
3,1301871,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M04,0,AAH
4,1301872,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M05,0,AAH
...,...,...,...,...,...,...,...
2074315,3905599,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2019M12,NaN,DCM
2074316,3905600,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M01,NaN,DCM
2074317,3905601,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M02,NaN,DCM
2074318,3905602,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M03,NaN,DCM


Now we are ready to merge the 2 datasets. Three new columns 'Country', 'Lat' and 'Long' are appended to the dataframe referencing the additional info on each foreign airport.

In [16]:
known_df[['Country', 'Lat', 'Long']] = pd.DataFrame(list(known_df["Airport_code"].map(reference_locations_dict)), index=known_df.index)

In [17]:
known_df

,index,Direction,Irish Airport,Foreign Airport,Month,DATA,Airport_code,Country,Lat,Long
0,1301868,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M01,0,AAH,GERMANY,50.823,6.187
1,1301869,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M02,0,AAH,GERMANY,50.823,6.187
2,1301870,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M03,0,AAH,GERMANY,50.823,6.187
3,1301871,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M04,0,AAH,GERMANY,50.823,6.187
4,1301872,Inward,"CFN Donegal (CFN),Republic Of Ireland","Aachen (AAH),Germany",2006M05,0,AAH,GERMANY,50.823,6.187
...,...,...,...,...,...,...,...,...,...,...
2074315,3905599,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2019M12,NaN,DCM,FRANCE,43.556,2.289
2074316,3905600,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M01,NaN,DCM,FRANCE,43.556,2.289
2074317,3905601,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M02,NaN,DCM,FRANCE,43.556,2.289
2074318,3905602,Outward,"WAT Waterford (WAT),Republic Of Ireland","Castres (DCM),France",2020M03,NaN,DCM,FRANCE,43.556,2.289


We can export this dataframe as a csv ready to be used in the dashboard.

In [18]:
#known_df.to_csv (r'C:\Users\ronan\PycharmProjects\Airport Project\assets\Airport_Traffic_Data.csv', index = False, header=True)